# Creación de funciones para endpoints

## Importamos librerías

In [2]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_games = pd.read_parquet("Datasets/output_steam_games.parquet") # Cargamos el dataset de juegos
df_games.head(2)

,app_name,price,item_id,developer,release_year,genres
0,Lost Summoner Kitty,4.99,761140,Kotoshiro,2018,Action
1,Ironbound,0.00,643980,Secret Level SRL,2018,Free to Play


In [4]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28899 entries, 0 to 28898
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   app_name      28899 non-null  object 
 1   price         28899 non-null  float64
 2   item_id       28899 non-null  int64  
 3   developer     28899 non-null  object 
 4   release_year  28899 non-null  int64  
 5   genres        28899 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.3+ MB


In [8]:
df_user_items = pd.read_parquet("Datasets/australian_users_items.parquet") # Cargamos el dataset de items
df_user_items.head(2)

,item_id,item_name,steam_id,items_count,user_id,playtime_hours
0,10,Counter-Strike,76561197970982479,277,76561197970982479,0.10
1,30,Day of Defeat,76561197970982479,277,76561197970982479,0.12


In [6]:
df_reviews = pd.read_parquet("Datasets/australian_user_reviews.parquet") # Cargamos el dataset de reviews
df_reviews.head(2)

,user_id,item_id,recommend,sentiment_analysis,posted_year
0,76561197970982479,1250,True,2,2011
1,76561197970982479,22200,True,1,2011


In [7]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59161 entries, 0 to 59160
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59161 non-null  object
 1   item_id             59161 non-null  int64 
 2   recommend           59161 non-null  bool  
 3   sentiment_analysis  59161 non-null  int64 
 4   posted_year         59161 non-null  int64 
dtypes: bool(1), int64(3), object(1)
memory usage: 1.9+ MB


In [5]:
df_games_and_reviews = pd.read_parquet("Datasets/games_and_reviews.parquet") # Cargamos el dataset de reviews
df_games_and_reviews.head(2)

,app_name,price,item_id,developer,release_year,genres,user_id,recommend,sentiment_analysis,posted_year
0,"Pirates, Vikings, and Knights II",0.0,17570,Octoshark Studios,2010,Action,76561198010653835,True,2,2015
1,"Pirates, Vikings, and Knights II",0.0,17570,Octoshark Studios,2010,Action,76561198075581042,True,2,2014


In [ ]:
df_reviews

### Función `PlayTimeGenre`

#### Función que devuelve el año con mas horas de juego para un género

In [42]:
def PlayTimeGenre(genero, df_games: pd.DataFrame, df_user_items: pd.DataFrame):
    # Convertimos el género proporcionado a minúsculas para una comparación sin distinción de mayúsculas
    genero = genero.lower()
    
    # Filtramos los juegos por género de manera más flexible
    df_filtered = df_games[df_games['genres'].str.lower().str.contains(fr'\b{genero}\b', na=False)]

    if not df_filtered.empty:
        # Convertimos 'item_id' a tipo de dato object en ambos DataFrames
        df_user_items['item_id'] = df_user_items['item_id'].astype(str)
        df_filtered['item_id'] = df_filtered['item_id'].astype(str)

        df_merged = pd.merge(df_user_items, df_filtered[['item_id', 'release_year']], left_on='item_id', right_on='item_id')
        
        # Verificamos si la longitud de df_filtered es mayor a cero, es decir, se encontraron géneros
        result = {"Año con más horas jugadas para el género {}: {}".format(genero.capitalize(), df_merged.groupby('release_year')['playtime_hours'].sum().idxmax())}
    else:
        result = {"Año con más horas jugadas para género {}: {}".format(genero.capitalize(), "Género no encontrado en la base de datos")}

    return result

In [47]:
PlayTimeGenre("Sports", df_games, df_user_items)

{'Año con más horas jugadas para el género Sports: 2015'}

### Función `UserForGenre`

#### Función que devuelve el usuario con más horas jugadas para un género 

In [51]:
def UserForGenre(genero: str, df_games: pd.DataFrame, df_user_items: pd.DataFrame):
    # Convertimos el género proporcionado a minúsculas para una comparación sin distinción de mayúsculas
    genero = genero.lower()
    
    # Filtramos los juegos por género de manera más flexible
    df_filtered = df_games[df_games['genres'].str.lower().str.contains(fr'\b{genero}\b', na=False)]
    
    if not df_filtered.empty:
        df_user_items['item_id'] = df_user_items['item_id'].astype(str)
        
        # Convertimos la columna 'item_id' en df_filtered a tipo 'str'
        df_filtered['item_id'] = df_filtered['item_id'].astype(str)
        
        df_merged = pd.merge(df_user_items, df_filtered[['item_id', 'release_year']], on='item_id')
        
        # Filtramos los datos desde el año 2003 en adelante
        df_merged = df_merged[df_merged['release_year'] >= 2003]
        user_with_most_playtime = df_merged.groupby('user_id')['playtime_hours'].sum().idxmax()
        playtime_by_year = df_merged.groupby(['release_year', 'user_id'])['playtime_hours'].sum().reset_index()
        playtime_by_year = playtime_by_year[playtime_by_year['user_id'] == user_with_most_playtime]
        playtime_by_year = playtime_by_year.rename(columns={'release_year': 'Año', 'playtime_hours': 'Horas'})
        
        # Convertimos las horas a enteros
        playtime_by_year['Horas'] = playtime_by_year['Horas'].astype(int)
        
        result = {
            "Usuario con más horas jugadas para Género {}:".format(genero.capitalize()): user_with_most_playtime,
            "Horas jugadas": [{"Año": str(row['Año']), "Horas": row['Horas']} for _, row in playtime_by_year.iterrows()]
        }
    else:
        result = {"Usuario con más horas jugadas para Género {}:".format(genero.capitalize()): "Género no encontrado en la base de datos", "Horas jugadas": []}
    
    return result


In [53]:
UserForGenre("Action", df_games, df_user_items)

{'Usuario con más horas jugadas para Género Action:': 'Sp3ctre',
 'Horas jugadas': [{'Año': '2003', 'Horas': 127},
  {'Año': '2004', 'Horas': 2123},
  {'Año': '2005', 'Horas': 355},
  {'Año': '2006', 'Horas': 14},
  {'Año': '2007', 'Horas': 1879},
  {'Año': '2008', 'Horas': 3},
  {'Año': '2009', 'Horas': 1805},
  {'Año': '2010', 'Horas': 1301},
  {'Año': '2011', 'Horas': 2581},
  {'Año': '2012', 'Horas': 6304},
  {'Año': '2013', 'Horas': 2007},
  {'Año': '2014', 'Horas': 2178},
  {'Año': '2015', 'Horas': 5236},
  {'Año': '2016', 'Horas': 488},
  {'Año': '2017', 'Horas': 722}]}

### Función `UsersRecommend`

#### Función que devuelve un Top 3 de los juegos mas recomendados por año

In [8]:
def UsersRecommend(año: int, df_games_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}

    # Filtramos por el año deseado
    df_filtered = df_games_and_reviews[df_games_and_reviews['release_year'] == año]

    # Filtramos por comentarios recomendados y sentiment_analysis positivo/neutral
    df_filtered = df_filtered[(df_filtered['recommend'] == True) & (df_filtered['sentiment_analysis'].isin([0, 1, 2]))]

    # Obtenemos el top 3 de juegos recomendados
    top_games = df_filtered['app_name'].value_counts().head(3).reset_index()
    top_games = top_games.rename(columns={'index': 'Puesto 1', 'app_name': 'Juego'})

    # Modificamos la estructura del resultado
    result = [{"Puesto {}".format(i + 1): juego, 'recomendaciones': count} for i, (juego, count) in enumerate(zip(top_games['Juego'], top_games['count']))]

    return result

In [13]:
UsersRecommend(2014, df_games_and_reviews)

[{'Puesto 1': 'Loadout', 'recomendaciones': 325},
 {'Puesto 2': 'Goat Simulator', 'recomendaciones': 254},
 {'Puesto 3': 'Insurgency', 'recomendaciones': 207}]

### Función `UsersWorstDeveloper`

#### Función que devuelve el Top 3 de peor desarrollador del año

In [57]:
def UsersWorstDeveloper(año: int, df_games_and_reviews: pd.DataFrame):
    # Verificamos si el año está dentro del rango esperado
    rango_aceptado = range(2010, 2016)
    if año not in rango_aceptado:
        return {"message": "Mi base de datos solo tiene registros entre 2010 y 2015"}

    # Filtramos por comentarios no recomendados y sentiment_analysis negativo
    df_filtered = df_games_and_reviews[(df_games_and_reviews['recommend'] == False) & (df_games_and_reviews['sentiment_analysis'] == 0)]

    # Filtramos por el año deseado
    df_filtered_year = df_filtered[df_filtered['release_year'] == año]

    # Si no hay datos para el año, retornamos mensaje
    if not df_filtered_year.empty:
        # Obtener los top 3 desarrolladores con menos recomendaciones
        top_developers = df_filtered_year['developer'].value_counts().head(3).reset_index()
        top_developers = top_developers.rename(columns={'index': 'Puesto 1', 'developer': 'Desarrollador'})

        # Modificamos la estructura del resultado
        result = [{"Puesto {}".format(i + 1): desarrollador} for i, desarrollador in enumerate(top_developers['Desarrollador'])]
    else:
        result = {"No hay juegos no recomendados para el año {}".format(año)}

    return result

In [58]:
UsersWorstDeveloper(2010, df_games_and_reviews)

{'message': 'Mi base de datos solo tiene registros entre 2010 y 2015'}

### Función `sentiment_analysis`

#### Función que devuelve el conteo de las reviews positivas, neutrales y negativas de una empresa desarrolladora

In [59]:
def sentiment_analysis(empresa_desarrolladora: str, df_games_and_reviews: pd.DataFrame):
    # Filtramos por desarrolladora
    df_filtered_developer = df_games_and_reviews[df_games_and_reviews['developer'] == empresa_desarrolladora]

    # Verificamos que haya datos para la desarrolladora
    if not df_filtered_developer.empty:
        # Contamos los sentimientos y mapeamos el número del sentimiento a su etiqueta correspondiente
        sentiment_counts = df_filtered_developer['sentiment_analysis'].value_counts().reset_index(drop=True)
        sentiment_mapping = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
        sentiment_counts.index = sentiment_counts.index.map(sentiment_mapping)
        
        result = {empresa_desarrolladora: sentiment_counts.to_dict()}
    else:
        result = 'No cuento con los registros de esa empresa en mi base de datos'

    return result

In [61]:
sentiment_analysis('Treyarch', df_games_and_reviews)

{'Treyarch': {'Negative': 155, 'Neutral': 121, 'Positive': 37}}